In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers


In [3]:
dataset_path = "."


In [4]:
image_size = (224, 224)
batch_size = 32


In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)


Found 36971 files belonging to 20 classes.
Using 29577 files for training.
Found 36971 files belonging to 20 classes.
Using 7394 files for validation.


In [8]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(num_classes)


20


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds   = val_ds.map(lambda x, y: (x / 255.0, y))

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [10]:
model = tf.keras.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])


In [11]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10


2025-12-27 10:51:12.004039: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:453] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 619 of 1000
2025-12-27 10:51:19.122531: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:483] Shuffle buffer filled.


925/925 ━━━━━━━━━━━━━━━━━━━━ 607s 636ms/step - accuracy: 0.3912 - loss: 1.9521 - val_accuracy: 0.8169 - val_loss: 0.5673
Epoch 2/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 583s 630ms/step - accuracy: 0.7321 - loss: 0.8212 - val_accuracy: 0.8680 - val_loss: 0.3920
Epoch 3/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 1294s 1s/step - accuracy: 0.8111 - loss: 0.5824 - val_accuracy: 0.9061 - val_loss: 0.2785
Epoch 4/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 546s 589ms/step - accuracy: 0.8471 - loss: 0.4520 - val_accuracy: 0.9080 - val_loss: 0.2773
Epoch 5/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 750s 810ms/step - accuracy: 0.8783 - loss: 0.3590 - val_accuracy: 0.9053 - val_loss: 0.2899
Epoch 6/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 636s 686ms/step - accuracy: 0.8971 - loss: 0.3076 - val_accuracy: 0.9268 - val_loss: 0.2177
Epoch 7/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 1236s 1s/step - accuracy: 0.9107 - loss: 0.2564 - val_accuracy: 0.9331 - val_loss: 0.2089
Epoch 8/10
925/925 ━━━━━━━━━━━━━━━━━━━━ 557s 601ms/step - accuracy: 0.9222 - loss: 0.2316 -

In [12]:
final_train_acc = history.history["accuracy"][-1]
final_val_acc = history.history["val_accuracy"][-1]

print("Final Training Accuracy:", final_train_acc)
print("Final Validation Accuracy:", final_val_acc)


NameError: name 'history' is not defined

In [13]:
model.save("crop_disease_cnn_model.keras")


In [14]:
import tensorflow as tf

model = tf.keras.models.load_model("crop_disease_mobilenet.keras")


In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "test_leaf.jpg"

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]

print("Predicted Disease:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
Predicted Disease: Strawberry___Leaf_scorch


In [19]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [20]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [21]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 266s 285ms/step - accuracy: 0.6388 - loss: 1.1755 - val_accuracy: 0.9088 - val_loss: 0.2977
Epoch 2/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 272s 295ms/step - accuracy: 0.8580 - loss: 0.4302 - val_accuracy: 0.9230 - val_loss: 0.2233
Epoch 3/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 280s 303ms/step - accuracy: 0.8826 - loss: 0.3497 - val_accuracy: 0.9370 - val_loss: 0.1868
Epoch 4/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 271s 293ms/step - accuracy: 0.8908 - loss: 0.3131 - val_accuracy: 0.9410 - val_loss: 0.1773
Epoch 5/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 274s 296ms/step - accuracy: 0.9041 - loss: 0.2834 - val_accuracy: 0.9413 - val_loss: 0.1628


In [22]:
model.save("crop_disease_mobilenet.keras")
